In [1]:
import csv
import os
from slugify import slugify

In [2]:
with open("bus_by_networks.csv", 'r') as in_file:
    tt = csv.DictReader(in_file)
    all_lines= list(tt)
#all_lines

In [3]:
all_networks = list(set([elem['agency_name'] for elem in all_lines]))
len(all_networks)

121

In [4]:
def create_settings_content(network_name, folder_name):
    content = """
#!/bin/bash

#
# set variales for analysis of network
#

PREFIX="{}"

OVERPASS_QUERY="http://overpass-api.de/api/interpreter?data=area[wikidata=Q13917][type=boundary];(rel(area)[route~'bus'];rel(br);rel[type='route'](r);)->.routes;(.routes;<<;rel(r.routes);way(r.routes);node(r.routes););out;"
NETWORK_LONG=""
NETWORK_SHORT="{}"

ANALYSIS_PAGE=""
ANALYSIS_TALK=""
#
# Routes data is in GitHub only, not in OSM-Wiki
#
WIKI_ROUTES_PAGE=""

ANALYSIS_OPTIONS="--check-access --check-bus-stop --check-name --check-osm-separator --check-sequence --check-stop-position --check-version --coloured-sketchline --check-motorway-link --max-error=10 --multiple-ref-type-entries=analyze --positive-notes"

# --language=en
# --check-bus-stop 
# --expect-network-short
# --expect-network-long
# --expect-network-short-for=
# --expect-network-long-for=
# --relaxed-begin-end-for=

    """.format(folder_name, network_name)
    return content


def create_lines_list(network_name, line_list):
    content = """
#
# This data is *not* stored in the OSM Wiki but in GitHub 
#
# Format: format is like in the OSM Wiki
#
# Links: [[...|...]] are interne link like in the OSM Wiki
#        [... ...] are external links
#
# Headers start with '=', '==', '===', '====', ... at the beginning of a line
#
# Simple text starts with '-' at the beginning of a line.
#    A single '-' at the beginning of a line, followed by nothing:
#    - if there was simple text before, it creates a line feed (i.e. encloses the text in a paragraph <p> ... </p>)
#    - if there was no simple text before or a line feed, it creates an empty line (i.e. <p>&amp;nbsp;</p>)
#
# !!!Text yellow background!!! in simple text or headers
# '''''Text mit bold and italics''''' in simple text or headers
# '''Text with bold chars''' in simple text or headers
# ''Text with italic chars'' in simple text or headers
#
# Comments start with '#' at the beginning of a line. '#' inside text is not recognized as the start of a comment, i.e.. '#' may occur inside of text.
#
# Format of the file: UTF-8
#
#
# Contents in CSV-Format - if ';' is part of the field, then enclose that field in "..."
#
# ref;type;comment;from;to;operator
#
# - ref       == tag 'ref' of route or route_master
#                250            defines that routes with 'ref'='250' are expected
#                250|250a|250b  defines that routes with 'ref'='250' or 'ref'='250a' or 'ref'='250b' are expected - independent of whether this is allowed according to PTv1/PTv2
# - type      == contents of tags 'route' respectively 'route_master'
# - comment   == can include comments like; Bus, Expressbus, ...  will not be analyzed, but simply be printed out
#                !Text with yellow background! in comment (surrounded by single !)
# - from      == if there is more than one entry with "ref;type" and "operator" is the same, then 'from' and 'to are also used to distinguish between same line number in different cities/villages
# - to        == if there is more than one entry with "ref;type" and "operator" is the same, then 'from' and 'to are also used to distinguish between same line number in different cities/villages
# - operator  == if there is more than one entry with "ref;type", then "operator" is used to distinguish between same line number in different cities/villages
#


= Overview on '''{}''' Public Transport Lines

-
- '''!!!This summary is a list of what has been published in open data by Île-de-France Mobilités and found in OSM!!!'''
-
- Feel free to modify the list on github
-
#
#ref;type;comment;from;to;operator

== Lines""".format(network_name)
    sorted_line_list = sorted(line_list, key=lambda k: k['route_short_name']) 
    for a_line in sorted_line_list:
        content += "\n{};{};;;;".format(a_line['route_short_name'], a_line['osm_transport_mode'])
    
    return content


In [6]:
for a_network in all_networks:
    if a_network in ['RATP', 'SNCF', 'SNCF/RATP']:
        continue
    lines = [elem for elem in all_lines if elem['agency_name'] == a_network]
    if len(lines) >= 10:
        print(a_network)
        folder_name = "FR-IDF-{}".format(slugify(a_network))
        folder_path = os.path.join(os.getcwd(),folder_name)
        os.makedirs(folder_path)

        content = create_settings_content(a_network, folder_name)
        with open(os.path.join(folder_path,"settings.sh"),"w") as setting_file:
            setting_file.write(content)
            
        content = create_lines_list(a_network, lines)
        file_name = "{}-Routes.txt".format(folder_name)
        with open(os.path.join(folder_path,file_name),"w") as route_file:
            route_file.write(content)

Pep's
CSO
Filéo
Pays de l'Ourcq
Phébus
Busval d'Oise
Seine-Saint-Denis
Paladin
Pays Créçois
TRAMY
Sol'R
R'bus
Houdanais
Aérial
Trans'Essonne
TRANSPORTS DANIEL MEYER
Arlequin
MELIBUS
Albatrans
SITUS
Pays de Meaux
Apolo 7
SQYBUS
HOURTOULE
ORMONT TRANSPORT
Noctilien
Rambouillet Interurbain
Goëlys
Bus en Seine
VyBus
Com'Bus
Mobicaps
STIVO
Réseau du Canton de Perthes
CIF
Pays Fertois
SAVAC
Sénart-Bus
Seine et Marne Express
PROCARS
Val de Seine
LACROIX
Entre Seine et Forêt
CARS MOREAU
TAM
LES CARS BLEUS
CEAT
Poissy Aval - Deux Rives de Seine
Seine Sénart Bus
Transdev Ile-de-France Conflans
STILL
Comète
Yerres
TICE
Parisis
Siyonne
